This notebook imports the cleaned GenBankextraction.csv created in GenBank_ExtractionScript.ipynb and creates a dataframe. It then splits up columns and adds additional columns based on original values. It also imports cataloguelife.csv, created in CatalogueLife_Distribution.ipynb, and merges the two dataframes for a more complete view of each GenBank record.

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
GenBank_file = 'polypodiales_GenBankextraction.csv'
CatalogueLife_file = 'poly_cataloguelife.csv'
merged_dataset = "poly_merged_dataset.csv"
final_dataset = "poly_final_dataset.csv"

In [3]:
#importing base spreadsheet and reading it into pandas 
infile_GenBank = open(GenBank_file, 'r')
df = pd.read_csv(infile_GenBank)
df.head()

,GenBank_ID,sci_name,organism_name,authors,article_title,article_extra,journal,journal_extra,doi,pubmed_ID,biosample,bioproject,specimen_voucher,collection_date,collected_by,location,coordinates,note,record_overlap
0,MH749231.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,missing,PRJNA389125,Sedaghatpour107,27-Oct-2010,"M. Sedaghatpour, K. S. Ingram, K. L. Faulconer...",USA: Regional Garden at the United States Bota...,38.8878252649 N 77.0146480452 W,psbA-trnH intergenic spacer region; may contai...,gb|MH749231.1| gi|1496297936
1,MH749155.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,missing,PRJNA389125,Sedaghatpour107,27-Oct-2016,"M. Sedaghatpour, K. S. Ingram, K. L. Faulconer...",USA: Regional Garden at the United States Bota...,38.8878252649 N 77.0146480452 W,missing,gb|MH749155.1| gi|1496297785
2,LC389082.1,Polystichum pseudomakinoi,Polystichum pseudomakinoi,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,missing,missing,missing,missing,missing,missing,missing,missing,dbj|LC389082.1| gi|1497396132
3,LC389081.1,Polystichum lepidocaulon,Polystichum lepidocaulon,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,missing,missing,missing,missing,missing,missing,missing,missing,dbj|LC389081.1| gi|1497396130
4,LC389080.1,Polystichum rigens,Polystichum rigens,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,missing,missing,missing,missing,missing,missing,missing,missing,dbj|LC389080.1| gi|1497396128


Modfying existing columns and adding derived columns to the dataframe.

In [4]:
#creating a new column based on columns pubmed_ID, journal, and article_extra that group records according to article linking
df_manip = df

df_manip.loc[(df_manip.pubmed_ID != 999), 'linking'] = 1 #article is directly hyperlinked to the GenBank record via pubmed
df_manip.loc[((df_manip.journal != 'Unpublished') | (df_manip.journal != 'missing')) & (df_manip.pubmed_ID == 999), 'linking'] = 2 #article citation exists in GenBank record, but it does not have a direct link
df_manip.loc[(df_manip.journal == 'Unpublished') & (df_manip.pubmed_ID == 999), 'linking'] = 3 #article title exists, potentially an indicator of future publication, but GenBank states that it is unpublished
df_manip.loc[(df.article_extra == 'missing') & (df_manip.journal_extra == 'missing') & (df_manip.pubmed_ID == 999), 'linking'] = 4 #no article information exists in GenBank-- it has been omitted

In [5]:
#splitting up organism_name to extract the common name of the record specimen
df_manip2 = df_manip

# grabbing the common name from the organism_name variable, located in parentheses
common_name = (df_manip2['organism_name'].str.replace(")", "").str.split("(", n = 1, expand = True))
df_manip2['common_name'] = common_name[1]
df_manip2['common_name'].fillna("missing", inplace=True)
df_manip2[df_manip2['common_name'] != 'missing']

,GenBank_ID,sci_name,organism_name,authors,article_title,article_extra,journal,journal_extra,doi,pubmed_ID,...,bioproject,specimen_voucher,collection_date,collected_by,location,coordinates,note,record_overlap,linking,common_name
186,MG593106.1,Myriopteris aurea,chloroplast Myriopteris aurea (golden cloak fern),"Ponce, M.M., Scataglini, M.A., Ponce, M.M., Sc...",Further progress towards the delimitation of C...,Direct Submission,"Org. Divers. Evol. 18 (2), 175-186 (2018)","Submitted (28-NOV-2017) CONICET, Instituto de ...",missing,999,...,missing,M. E. Mulgura et al. 4227 (SI),missing,missing,missing,missing,missing,gb|MG593106.1| gi|1489424447,2.0,golden cloak fern
323,MH019744.1,Adiantum raddianum,plastid Adiantum raddianum (delta maidenhair f...,"Huiet, L., Li, F.-W., Kao, T.-T., Prado, J., S...",A worldwide phylogeny of Adiantum (Pteridaceae...,Direct Submission,"Taxon 67 (3), 488-502 (2018)","Submitted (03-MAR-2018) Biology, Duke Universi...",missing,999,...,missing,DUKE Prado 2148,missing,missing,Brazil,missing,RNA polymerase alpha subunit; translated via R...,gb|MH019744.1| gi|1485740400,2.0,delta maidenhair fern
442,MH019624.1,Adiantum raddianum,plastid Adiantum raddianum (delta maidenhair f...,"Huiet, L., Li, F.-W., Kao, T.-T., Prado, J., S...",A worldwide phylogeny of Adiantum (Pteridaceae...,Direct Submission,"Taxon 67 (3), 488-502 (2018)","Submitted (03-MAR-2018) Biology, Duke Universi...",missing,999,...,missing,DUKE Prado 2148,missing,missing,Brazil,missing,missing,gb|MH019624.1| gi|1485740252,2.0,delta maidenhair fern
558,MH019508.1,Adiantum raddianum,plastid Adiantum raddianum (delta maidenhair f...,"Huiet, L., Li, F.-W., Kao, T.-T., Prado, J., S...",A worldwide phylogeny of Adiantum (Pteridaceae...,Direct Submission,"Taxon 67 (3), 488-502 (2018)","Submitted (03-MAR-2018) Biology, Duke Universi...",missing,999,...,missing,DUKE Prado 2148,missing,missing,Brazil,missing,missing,gb|MH019508.1| gi|1485740020,2.0,delta maidenhair fern
666,MH019398.1,Adiantum raddianum,plastid Adiantum raddianum (delta maidenhair f...,"Huiet, L., Li, F.-W., Kao, T.-T., Prado, J., S...",A worldwide phylogeny of Adiantum (Pteridaceae...,Direct Submission,"Taxon 67 (3), 488-502 (2018)","Submitted (03-MAR-2018) Biology, Duke Universi...",missing,999,...,missing,DUKE Prado 2148,missing,missing,Brazil,missing,ATP synthase CF1 alpha chain; translated via R...,gb|MH019398.1| gi|1485739829,2.0,delta maidenhair fern
855,KY806705.1,Arachniodes aristata,chloroplast Arachniodes aristata (Indian holly...,"Hori, K., Okuyama, Y., Watano, Y., Murakami, N...",Recurrent hybridization without homoeologous c...,Direct Submission,"Chromosom Bot 13 (1), 9-24 (2018)","Submitted (23-MAR-2017) Makino Herbarium, Toky...",missing,999,...,missing,missing,missing,missing,missing,missing,missing,gb|KY806705.1| gi|1356686690,2.0,Indian holly fern
902,KY806658.1,Arachniodes aristata,Arachniodes aristata (Indian holly fern),"Hori, K., Okuyama, Y., Watano, Y., Murakami, N...",Recurrent hybridization without homoeologous c...,Direct Submission,"Chromosom Bot 13 (1), 9-24 (2018)","Submitted (23-MAR-2017) Makino Herbarium, Toky...",missing,999,...,missing,missing,missing,missing,missing,missing,similar to glyceraldehyde-3-phosphate dehydrog...,gb|KY806658.1| gi|1356686642,2.0,Indian holly fern
919,KY794840.1,Arachniodes aristata,Arachniodes aristata (Indian holly fern),"Hori, K., Okuyama, Y., Watano, Y., Murakami, N...",Recurrent hybridization without homoeologous c...,Direct Submission,"Chromosom Bot 13 (1), 9-24 (2018)","Submitted (22-MAR-2017) Makino Herbarium, Toky...",missing,999,...,missing,missing,missing,missing,missing,missing,missing,gb|KY794840.1| gi|1315908970,2.0,Indian holly fern
920,KY794839.1,Arachniodes aristata,Arachniodes aristata (Indian holly fern),"Hori, K., Okuyama, Y., Watano, Y., Murakami, N...",Recurrent hybridization without homoeologous c...,Direct Submission,"Chromosom Bot 13 (1), 9-24 (2018)","Submitted (22-MAR-2017) Makino Herbarium, Toky...

In [6]:
# splitting up location to isolate country from more granular textual data to prepare for geopolitcal region matching
location_info = df_manip2['location'].str.split(":", n=1, expand=True)
df_manip2['location_country'] = location_info[0]
df_manip2['location_specific'] = location_info[1]
df_manip2['location_country'].fillna("missing", inplace=True)
df_manip2['location_specific'].fillna("missing", inplace=True)
# df_manip2[df_manip2['location_specific'] != "missing"]

In [7]:
#creating a new column based on columns coordinates and location_country that group records according to location granularity
df_manip3 = df_manip2

# grouping location data by granuality
df_manip3.loc[(df_manip3.coordinates != "missing"), 'locality_specificity'] = 3   #most granular, contains coordinates
df_manip3.loc[(df_manip3.location_country != "missing") & (df_manip3.location_specific != "missing") & (df_manip3.coordinates == "missing"), 'locality_specificity'] = 2 #middle granular, specific location
df_manip3.loc[(df_manip3.location_country != "missing") & (df_manip3.location_specific == "missing") & (df_manip3.coordinates == "missing"), 'locality_specificity'] = 1 #least granular, country level
df_manip3.loc[(df_manip3.location_country == "missing") & (df_manip3.location_specific == "missing") & (df_manip3.coordinates == "missing"), 'locality_specificity'] = 999 #no locality information

# df_manip3[df_manip3['locality_specificity'].isnull()]
df_manip3

,GenBank_ID,sci_name,organism_name,authors,article_title,article_extra,journal,journal_extra,doi,pubmed_ID,...,collected_by,location,coordinates,note,record_overlap,linking,common_name,location_country,location_specific,locality_specificity
0,MH749231.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,...,"M. Sedaghatpour, K. S. Ingram, K. L. Faulconer...",USA: Regional Garden at the United States Bota...,38.8878252649 N 77.0146480452 W,psbA-trnH intergenic spacer region; may contai...,gb|MH749231.1| gi|1496297936,3.0,missing,USA,Regional Garden at the United States Botanic ...,3.0
1,MH749155.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,...,"M. Sedaghatpour, K. S. Ingram, K. L. Faulconer...",USA: Regional Garden at the United States Bota...,38.8878252649 N 77.0146480452 W,missing,gb|MH749155.1| gi|1496297785,3.0,missing,USA,Regional Garden at the United States Botanic ...,3.0
2,LC389082.1,Polystichum pseudomakinoi,Polystichum pseudomakinoi,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,missing,missing,dbj|LC389082.1| gi|1497396132,1.0,missing,missing,missing,999.0
3,LC389081.1,Polystichum lepidocaulon,Polystichum lepidocaulon,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,missing,missing,dbj|LC389081.1| gi|1497396130,1.0,missing,missing,missing,999.0
4,LC389080.1,Polystichum rigens,Polystichum rigens,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,missing,missing,dbj|LC389080.1| gi|1497396128,1.0,missing,missing,missing,999.0
5,LC389079.1,Polystichum makinoi,Polystichum makinoi,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,missing,missing,dbj|LC389079.1| gi|1497396126,1.0,missing,missing,missing,999.0
6,LC389078.1,Polystichum fibrillosopaleaceum,Polystichum fibrillosopaleaceum,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,missing,missing,dbj|LC389078.1| gi|1497396124,1.0,missing,missing,missing,999.0
7,LC389077.1,Polystichum longifrons,Polystichum longifrons,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,missing,missing,dbj|LC389077.1| gi|1497396122,1.0,missing,missing,missing,999.0
8,LC389076.1,Polystichum pseudomakinoi,Polystichum pseudomakinoi,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,

Now creating a dictionary to match the location with geopolitcal regions

In [8]:
geo_dict = {"Northern Africa": {"Algeria", "Egypt", "Libya", "Morocco", "Sudan", "Tunisia", "Western Sahara"},
                    "Eastern Africa": {"British Indian Ocean Territory", "Burundi", "Comoros", "Djibouti", "Eritrea",
                                       "Ethiopia", "French Southern Territories", "Kenya", "Madagascar", "Malawi",
                                       "Mauritius", "Mayotte", "Mozambique", "La Runion", "Rwanda", "Seychelles", "Somalia",
                                       "South Sudan", "Uganda", "United Republic of Tanzania", "Tanzania", "Zambia", "Zimbabwe", 
                                       "Reunion"},
                    "Middle Africa": {"Angola", "Cameroon", "Central African Republic", "Chad", "Congo",
                                      "Democratic Republic of the Congo", "Equatorial Guinea", "Equatorial", "Guinea", "Gabon",
                                      "Sao Tome and Principe"},
                    "Southern Africa": {"Botswana", "Eswatini", "Lesotho", "Namibia", "South Africa"},
                    "Western Africa": {"Benin", "Burkina Faso", "Cabo Verde", "Côte d'Ivoire", "Gambia", "Ghana", "Guinea",
                                       "Guinea-Bissau", "Liberia", "Mali", "Mauritania", "Niger", "Nigeria", "Saint Helena",
                                       "Senegal", "Sierra Leone", "Togo"},
                    "Caribbean": {"Anguilla", "Antigua", "Aruba", "Bahamas", "Barbados", "Bonaire",
                                  "British Virgin Islands", "Cayman Islands", "Cuba", "Curacao", "Dominica", "Dominican Republic",
                                  "Grenada", "Guadeloupe", "Haiti", "Jamaica", "Martinique", "Montserrat", "Puerto Rico",
                                  "Saint Barthélemy", "Saint Kitts & Nevis", "Saint Lucia", "Saint Martin",
                                  "Saint Vincent and the Grenadines", "Sint Maarten", "Saba", "Trinidad", "Tobago", "Trinidad and Tobago",
                                  "Turks & Caicos Islands", "Virgin Islands"},
                    "Central America": {"Belize", "Costa Rica", "Isla del Coco", "El Salvador", "Guatemala", "Honduras",
                                        "Mexico", "Nicaragua", "Panama"},
                    "South America": {"Argentina", "Bolivia", "Bouvet Island", "Brazil", "Chile", "Colombia", "Ecuador",
                                      "Falkland Islands", "Falkland Islands (Islas Malvinas)", "French Guiana", "Guyana", "Paraguay",
                                      "Peru", "South Georgia", "South Sandwich Islands", "Suriname", "Uruguay", "Venezuela"},
                    "Northern America": {"Bermuda", "Canada", "Greenland", "Saint Pierre & Miquelon", "USA", "Antarctica",
                                         "French Southern and Antarctic Lands", "Kerguelen Archipelago"},
                    "Central Asia": {"Kazakhstan", "Kyrgyzstan", "Tajikistan", "Turkmenistan", "Uzbekistan"},
                    "Eastern Asia": {"China", "Hong Kong", "Japan", "Macao", "Mongolia", "North Korea", "South Korea", "Korea", "Taiwan"},
                    "Southeastern Asia": {"Borneo","Brunei", "Cambodia", "Indonesia", "Laos", "Lesser Sunda Islands",
                                          "Malaysia", "Moluccas", "Myanmar", "Philippines", "Singapore", "Thailand",
                                          "Vietnam", "Viet Nam"},
                    "Southern Asia": {"Afghanistan", "Bangladesh", "Bhutan", "India", "Iran", "Maldives", "Nepal",
                                      "Pakistan", "Sri Lanka"},
                    "Western Asia": {"Armenia", "Azerbaijan", "Bahrain", "Cyprus", "Georgia", "Iraq", "Israel", "Jordan",
                                     "Kuwait", "Lebanon", "Oman", "Qatar", "Saudi Arabia", "Palestine", "Syria", "Turkey",
                                     "United Arab Emirates", "Yemen"},
                    "Eastern Europe": {"Belarus", "Bulgaria", "Czech Republic", "Hungary", "Poland", "Moldavia", "Moldova",
                                       "Romania", "Russia", "Russian Far East", "European Russia", "Slovakia", "Ukraine"},
                    "Northern Europe": {"Guernsey", "Jersey", "Sark", "Denmark", "Estonia", "Faroe Islands", "Finland",
                                        "Iceland", "Ireland", "Isle of Man", "Latvia", "Lithuania", "Norway", "Svalbard",
                                        "Jan Mayen", "Sweden", "United Kingdom", "Great Britain", "Ireland", "Scotland",
                                        "Wales", "Aland Islands"},
                    "Southern Europe": {"Albania", "Andorra", "Bosnia & Herzegovina", "Bosnia & Hercegovina", "Croatia", "Gibraltar", "Greece",
                                        "Holy See", "Italy", "Malta", "Montenegro", "Portugal", "San Marino", "Serbia",
                                        "Slovenia", "Spain", "former Yugoslavia", "Yugoslavia", "Macedonia"},
                    "Western Europe": {"Austria", "Belgium", "France", "Germany", "Liechtenstein", "Luxembourg", "Monaco",
                                       "Netherlands", "Switzerland"},
                    "Australia & New Zealand": {"Australia", "Christmas Island", "Cocos (Keeling) Islands", "Heard Island",
                                               "McDonald Islands", "New Zealand", "Norfolk Island"},
                    "Melanesia, Micronesia & Polynesia": {"Fiji", "New Caledonia", "Papua New Guinea", "New Guinea",
                                                          "Solomon Islands", "Vanuatu", "Guam", "Kiribati",
                                                          "Marshall Islands", "Micronesia", "Nauru", "Northern Marianas",
                                                          "Marianas Island", "Southern Marianas", "Palau",
                                                          "United States Minor Outlying Islands", "American Samoa",
                                                          "Western Samoa", "Cook Islands","French Polynesia", "Niue",
                                                          "Pitcairn", "Samoa", "Tokelau", "Tonga", "Tuvalu",
                                                          "Wallis & Futuna Islands"},
                   "Undefined": {"Indian Ocean"},
                   "missing": {"missing"}}

Dictionary values with judgement calls:
- Tanzania                              | Eastern Africa  
- Congo                                 | Eastern Africa
- Viet Nam                              | Southeastern Asia
- Reunion                               | Eastern Africa (close to Madagascar, which classified as Eastern Africa)
- Falkland Islands (Islas Malvinas)     | South America
- French Southern and Antarctic Lands   | Northern America (Antarctica classified as Northern America)
- Equatorial Guinea                     | Middle Africa
- Hong Kong                             | Eastern Asia
- Kerguelen Archipelago                 | Northern America (same as French Southern and Antarctic Lands)
- Trinidad and Tobago                   | Caribbean
- Indian Ocean                          | *Undefined*
- Korea                                 | Eastern Asia

In [9]:
#create dictionary that sets countries as keys and geopolitical regions as values
geo = {}

for g, countrylist in geo_dict.items():
    for country in countrylist:
        geo[country] = g

In [10]:
#matching each country in location_country to a geopolitical region
df_manip4 = df_manip3

df_manip4['geopolitical_match'] = [geo[v.strip()] for v in df_manip4['location_country']]
df_manip4.head()

,GenBank_ID,sci_name,organism_name,authors,article_title,article_extra,journal,journal_extra,doi,pubmed_ID,...,location,coordinates,note,record_overlap,linking,common_name,location_country,location_specific,locality_specificity,geopolitical_match
0,MH749231.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,...,USA: Regional Garden at the United States Bota...,38.8878252649 N 77.0146480452 W,psbA-trnH intergenic spacer region; may contai...,gb|MH749231.1| gi|1496297936,3.0,missing,USA,Regional Garden at the United States Botanic ...,3.0,Northern America
1,MH749155.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,...,USA: Regional Garden at the United States Bota...,38.8878252649 N 77.0146480452 W,missing,gb|MH749155.1| gi|1496297785,3.0,missing,USA,Regional Garden at the United States Botanic ...,3.0,Northern America
2,LC389082.1,Polystichum pseudomakinoi,Polystichum pseudomakinoi,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,missing,dbj|LC389082.1| gi|1497396132,1.0,missing,missing,missing,999.0,missing
3,LC389081.1,Polystichum lepidocaulon,Polystichum lepidocaulon,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,missing,dbj|LC389081.1| gi|1497396130,1.0,missing,missing,missing,999.0,missing
4,LC389080.1,Polystichum rigens,Polystichum rigens,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,missing,missing,missing,dbj|LC389080.1| gi|1497396128,1.0,missing,missing,missing,999.0,missing


Now importing the supplemental Catalogue of Life merged dataset, and merging it with the current GenBank (df_manip4) dataframe.

In [11]:
#importing supplemental dataset and reading it into pandas 
infile_CatLife = open(CatalogueLife_file, 'r')
df_supp = pd.read_csv(infile_CatLife)
df_supp.drop(columns=['Unnamed: 0'], inplace=True)
df_supp.head()

,sci_name,vernacularName,geopolitical_regions,location_distribution
0,Bolbitis novoguineensis,NaN,"['Melanesia, Micronesia & Polynesia']",['New Guinea']
1,Bolbitis occidentalis,NaN,['South America'],['Ecuador']
2,Acrostichum pandurifolium,NaN,NaN,NaN
3,Gymnopteris pandurifolia,NaN,NaN,NaN
4,Leptochilus pandurifolius,NaN,NaN,NaN


In [12]:
merged = pd.merge(df_manip4, df_supp, how="left", on='sci_name')
merged.head()

,GenBank_ID,sci_name,organism_name,authors,article_title,article_extra,journal,journal_extra,doi,pubmed_ID,...,record_overlap,linking,common_name,location_country,location_specific,locality_specificity,geopolitical_match,vernacularName,geopolitical_regions,location_distribution
0,MH749231.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,...,gb|MH749231.1| gi|1496297936,3.0,missing,USA,Regional Garden at the United States Botanic ...,3.0,Northern America,NaN,['Northern America'],['USA;Canada;Greenland']
1,MH749155.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,...,gb|MH749155.1| gi|1496297785,3.0,missing,USA,Regional Garden at the United States Botanic ...,3.0,Northern America,NaN,['Northern America'],['USA;Canada;Greenland']
2,LC389082.1,Polystichum pseudomakinoi,Polystichum pseudomakinoi,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,dbj|LC389082.1| gi|1497396132,1.0,missing,missing,missing,999.0,missing,NaN,['Eastern Asia'],['China;Japan;South Korea']
3,LC389081.1,Polystichum lepidocaulon,Polystichum lepidocaulon,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,dbj|LC389081.1| gi|1497396130,1.0,missing,missing,missing,999.0,missing,NaN,['Eastern Asia'],['China;Taiwan;Japan;South Korea']
4,LC389080.1,Polystichum rigens,Polystichum rigens,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,dbj|LC389080.1| gi|1497396128,1.0,missing,missing,missing,999.0,missing,NaN,['Eastern Asia'],['China;Japan;South Korea']


In [13]:
#exporting untouched merged dataset, as proof of concept of variables
merged.to_csv(merged_dataset)

Finally, the merged dataset is rearranged and the unnecessary columns are dropped. The final version is exported out as a .csv.

In [14]:
#merging common_name and vernacularName where common_name values are used when vernacularName is blank

merged.vernacularName.fillna(merged.common_name, inplace=True)
merged.head()
# merged[merged['vernacularName'] != "missing"]  # checking common_name against vernacular name for override

,GenBank_ID,sci_name,organism_name,authors,article_title,article_extra,journal,journal_extra,doi,pubmed_ID,...,record_overlap,linking,common_name,location_country,location_specific,locality_specificity,geopolitical_match,vernacularName,geopolitical_regions,location_distribution
0,MH749231.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,...,gb|MH749231.1| gi|1496297936,3.0,missing,USA,Regional Garden at the United States Botanic ...,3.0,Northern America,missing,['Northern America'],['USA;Canada;Greenland']
1,MH749155.1,Dryopteris marginalis,chloroplast Dryopteris marginalis,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Direct Submission,Unpublished,Submitted (15-AUG-2018) National Museum of Nat...,missing,999,...,gb|MH749155.1| gi|1496297785,3.0,missing,USA,Regional Garden at the United States Botanic ...,3.0,Northern America,missing,['Northern America'],['USA;Canada;Greenland']
2,LC389082.1,Polystichum pseudomakinoi,Polystichum pseudomakinoi,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,dbj|LC389082.1| gi|1497396132,1.0,missing,missing,missing,999.0,missing,missing,['Eastern Asia'],['China;Japan;South Korea']
3,LC389081.1,Polystichum lepidocaulon,Polystichum lepidocaulon,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,dbj|LC389081.1| gi|1497396130,1.0,missing,missing,missing,999.0,missing,missing,['Eastern Asia'],['China;Taiwan;Japan;South Korea']
4,LC389080.1,Polystichum rigens,Polystichum rigens,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,Direct Submission,"Molecules 23 (8), E1843 (2018)",Submitted (19-JUN-2018) Contact:Junichi Shinoz...,10.3390/molecules23081843,30042370,...,dbj|LC389080.1| gi|1497396128,1.0,missing,missing,missing,999.0,missing,missing,['Eastern Asia'],['China;Japan;South Korea']


In [15]:
# selecting only relevant columns for final dataset, renaming for clarity, and changing data type from float to int
final = merged[['GenBank_ID', 'sci_name', 'vernacularName', 'authors', 'article_title', 'journal', 'doi', 'pubmed_ID', 'linking',
              'specimen_voucher', 'collection_date', 'collected_by', 'location_country', 'location_specific',
              'coordinates', 'locality_specificity', 'geopolitical_match', 'geopolitical_regions', 'location_distribution']]
final.rename(columns={'vernacularName': 'common_name', 'geopolitical_regions': 'geopolitical_distribution'}, inplace=True)
final = final.astype({"linking": int, "locality_specificity": int})
final

C:\Users\daria\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,GenBank_ID,sci_name,common_name,authors,article_title,journal,doi,pubmed_ID,linking,specimen_voucher,collection_date,collected_by,location_country,location_specific,coordinates,locality_specificity,geopolitical_match,geopolitical_distribution,location_distribution
0,MH749231.1,Dryopteris marginalis,missing,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Unpublished,missing,999,3,Sedaghatpour107,27-Oct-2010,"M. Sedaghatpour, K. S. Ingram, K. L. Faulconer...",USA,Regional Garden at the United States Botanic ...,38.8878252649 N 77.0146480452 W,3,Northern America,['Northern America'],['USA;Canada;Greenland']
1,MH749155.1,Dryopteris marginalis,missing,"Zuniga, J.D., Mulcahy, D.G., Zuniga, J.D.",Barcodes from the Global Genome Initiative for...,Unpublished,missing,999,3,Sedaghatpour107,27-Oct-2016,"M. Sedaghatpour, K. S. Ingram, K. L. Faulconer...",USA,Regional Garden at the United States Botanic ...,38.8878252649 N 77.0146480452 W,3,Northern America,['Northern America'],['USA;Canada;Greenland']
2,LC389082.1,Polystichum pseudomakinoi,missing,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,"Molecules 23 (8), E1843 (2018)",10.3390/molecules23081843,30042370,1,missing,missing,missing,missing,missing,missing,999,missing,['Eastern Asia'],['China;Japan;South Korea']
3,LC389081.1,Polystichum lepidocaulon,missing,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,"Molecules 23 (8), E1843 (2018)",10.3390/molecules23081843,30042370,1,missing,missing,missing,missing,missing,missing,999,missing,['Eastern Asia'],['China;Taiwan;Japan;South Korea']
4,LC389080.1,Polystichum rigens,missing,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,"Molecules 23 (8), E1843 (2018)",10.3390/molecules23081843,30042370,1,missing,missing,missing,missing,missing,missing,999,missing,['Eastern Asia'],['China;Japan;South Korea']
5,LC389079.1,Polystichum makinoi,missing,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,"Molecules 23 (8), E1843 (2018)",10.3390/molecules23081843,30042370,1,missing,missing,missing,missing,missing,missing,999,missing,['Undefined;Southern Asia;Eastern Asia'],['China;Tibet;South Korea;Japan;;Bhutan']
6,LC389078.1,Polystichum fibrillosopaleaceum,missing,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,"Molecules 23 (8), E1843 (2018)",10.3390/molecules23081843,30042370,1,missing,missing,missing,missing,missing,missing,999,missing,['Eastern Asia'],['Japan;South Korea']
7,LC389077.1,Polystichum longifrons,missing,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,"Molecules 23 (8), E1843 (2018)",10.3390/molecules23081843,30042370,1,missing,missing,missing,missing,missing,missing,999,missing,['Eastern Asia'],['Japan']
8,LC389076.1,Polystichum pseudomakinoi,missing,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,"Molecules 23 (8), E1843 (2018)",10.3390/molecules23081843,30042370,1,missing,missing,missing,missing,missing,missing,999,missing,['Eastern Asia'],['China;Japan;South Korea']
9,LC389075.1,Polystichum lepidocaulon,missing,"Shinozaki, J., Nakene, T., Takano, A., Shinoza...",Squalene Cyclases and Cycloartenol Synthases f...,"Molecules 23 (8), E1843 (2018)",10.3390/molecules23081843,30042370,1,missing,missing,missing,missing,missing,missing,999,missing,['Eastern Asia'],['China;Taiwan;Japan;South Korea']


In [16]:
#exporting final dataset
final.to_csv(final_dataset)